In [1]:
from utils_filter import *

Subsets outlooks, pph, and report data in various ways (add label to each datapoint for each method of subsetting so can be pulled out of full dataset later)


In [ ]:
data_location = 'data'
moderate = True # only consider moderate days
outlooks, pph, reports = read_datasets(data_location, moderate)

Subset by threshold

Subset by ramp up/down

Subset by accurate/inaccurate

Consider forecast issue time for day 1--subset reports (and revise pph?)--look at other studies to see how they handled multiple day 1 outlooks (see the one in slack). Starting point should be use first forecasts, later ones would be a different analysis for another time.

Subset by season

Subset by region